In [1]:
import math
import copy
from sympy import Symbol, solve

Fy   = 2.75000e-01
E    = 2.10000e+02
u    = 0.3
G    = E/(2*(1+u))

ColumnForceData = [20.805, -8307, -10615, -3.0394, -6.7290]         # Pu (Positive in Compression), Mux, Muy, Vux, Vuy - 기둥 Nodal Output,
GirderForceData = [50.05, 7518, 7308, 50, 100]                      # Pu (Positive in Compression), Mux, Muy, Vux, Vuy - Y방향 큰보 Nodal Output,
BeamForceData = [50.05, 7518, 7308, 50, 100]                        # Pu (Positive in Compression), Mux, Muy, Vux, Vuy - Y방향 작은보 Nodal Output
SubColumnForceData = [0, 20000, 20000, 50, 100]                    # Pu (Positive in Compression), Mux, Muy, Vux, Vuy - 샛기둥 Nodal Output
                                          
ColumnLength = 7000                                                                    # Floor가 있는 층이 있으면 강축방향 Brace를 층에서 잡아줌 / 부재별 Length로 사용가능
ColumnUnbracedSegment = 2                                                             # 기둥에 추가 Lateral Brace 개수 존재 여부 (약축 Unbraced Length 잡아줌, 1은 Lateral Brace 없는 것)
GirderLength = 7000                                                                    # Girder & Beam 강축방향 Unbraced Length / 부재별 Length로 사용가능
GirderUnbracedSegment = 2                                                             # X-Girder 약축방향 Lateral Brace 개수 존재 여부 (약축 Unbraced Length 잡아줌, 1은 Lateral Brace 없는 것, 0은 Continuous Braced)
BeamLength = 5000                                                                      # Beam 길이
BeamAttachedGirderLength = 5500                                                      # Beam이 붙은 Girder의 길이

K = 1

In [2]:
HsectionKeys = [
    'SectionName', 'Shape', 'D', 'W', 'tw', 'tf', 'radi', 'Area', 'Sxx', 'Syy', 'Zxx', 'Zyy', 'Ixx', 'Iyy', 'rx', 'ry', 'J', 'Cwp',
    ]

LsectionKeys = [
    'SectionName', 'Shape', 'D', 'W', 'tw', 'tf', 'radi', 'Area', 'Sxx', 'Syy', 'Zxx', 'Zyy', 'Ixx', 'Iyy', 'Ixy', 'rx', 'ry', 'J', 'Cwp', 'theta', 'xc', 'yc', 'Iu', 'Iv', 'Sumin', 'Svmin', 'ru', 'rv'
    ]

TsectionKeys = [
    'SectionName', 'Shape', 'D', 'W', 'tw', 'tf', 'radi', 'Area', 'Sxx', 'Syy', 'Zxx', 'Zyy', 'Ixx', 'Iyy', 'rx', 'ry', 'J', 'Cwp', 'xc', 'yc'
    ]

DoubleLsectionKeys = [
    'SectionName', 'Shape', 'D', 'W', 'tw', 'tf', 'radi', 'Gap', 'Area', 'Sxx', 'Syy', 'Zxx', 'Zyy', 'Ixx', 'Iyy', 'rx', 'ry', 'J', 'Cwp', 'xc', 'yc'
    ]

forceKeys = [
    'Pu', 'Mux', 'Muy', 'Vux', 'Vuy'
    ]

In [3]:
# 0             1       2  3  4   5   6     7
# Section Name, Shape,  D, W, tw, tf, radi, Gap

Section = [
['L-90x90x8',           'L',              90,               90,               8,          8,                0],
['CT-305x305x75',       'T',              304.8,            304.8,            11.9,       19.7,             0],
['2L-90x90x8',          '2L',             90,               90,               8,          8,                0,          10],
['UB-203x102x23',	      'H',              203.2,	      101.6,	      7.6,  	9.3,              0],
      ]


In [4]:
def Force_Dict_Composing(TargetList, KeyList) :
    Dict = dict(zip(KeyList, TargetList))
    return Dict

def get_key_by_value(Dict, TargetValue):
    for key, value in Dict.items():
        if value == TargetValue:
            return key
        
def H_Section_Prop_Calc(SectionPropList) :
    D = SectionPropList[2]
    W = SectionPropList[3]
    tw = SectionPropList[4]
    tf = SectionPropList[5]
    radi = SectionPropList[6]

    A = W*tf*2+D*tw-tw*tf*2
    Ixxc = ((W*tf**3+tw*(D/2-tf)**3)/12+W*tf*(D/2-tf/2)**2+tw*(D/2-tf)*((D/2-tf)/2)**2)*2
    Iyyc = ((tf*(W/2)**3*2+(D-2*tf)*(tw/2)**3)/12+tf*(W/2)*(W/4)**2*2+(D-2*tf)*(tw/2)*(tw/4)**2)*2
    Sxx = Ixxc/(D/2)
    Syy = Iyyc/(W/2)
    Zxx = (W*tf*(D/2-tf/2)+tw*(D/2-tf)*((D/2-tf)/2))*2
    Zyy = (tf*(W/2)*(W/4)*2+(D-2*tf)*(tw/2)*(tw/4))*2
    rx = math.sqrt(Ixxc/A)
    ry = math.sqrt(Iyyc/A)
    J = (2*W*tf**3+(D-tf)*tw**3)/3
    Cwp = Iyyc*(D-tf)**2/4

    SectionPropList.append(A)
    SectionPropList.append(Sxx)
    SectionPropList.append(Syy)
    SectionPropList.append(Zxx)
    SectionPropList.append(Zyy)
    SectionPropList.append(Ixxc)
    SectionPropList.append(Iyyc)
    SectionPropList.append(rx)
    SectionPropList.append(ry)
    SectionPropList.append(J)
    SectionPropList.append(Cwp)

    return SectionPropList

def L_Section_Prop_Calc(SectionPropList) :
    D = SectionPropList[2]
    W = SectionPropList[3]
    tw = SectionPropList[4]
    tf = SectionPropList[5]
    radi = SectionPropList[6]

    if D >= W :
        D = D
        W = W
    else : 
        D_temp = D
        tw_temp = tw
        W_temp = W
        tf_temp = tf
        SectionPropList[2] = W_temp
        SectionPropList[4] = tf_temp
        SectionPropList[3] = D_temp
        SectionPropList[5] = tw_temp
        D = W_temp
        W = D_temp
        tw = tf_temp
        tf = tw_temp

    A = W*tf+D*tw-tw*tf
    xc = (W**2*tf+tw**2*D-tw**2*tf)/(2*A)
    yc = (tw*D**2+W*tf**2-tw*tf**2)/(2*A)
    Ixxc = ((W*yc**3)-((W-tw)*(yc-tf)**3)+(tw*(D-yc)**3))/3
    Iyyc = ((D*xc**3)-((D-tf)*(xc-tw)**3)+(tf*(W-xc)**3))/3
    Ixyc = ((W*D)**2)/4-(W-tw)*(D-tf)*(tw+(W-tw)/2)*(tf+(D-tf)/2)-(A*xc*yc)
    Sxx = Ixxc/(D-yc)
    Syy = Iyyc/(W-xc)
    Zxx = tf*(D-(A/(2*W)))**2/2+W*(A/(2*W))**2/2+(W-tw)*((A/(2*W))-tw)**2/2
    Zyy = tf*(W-(A/(2*D)))**2/2+D*(A/(2*D))**2/2+(D-tf)*((A/(2*D))-tf)**2/2
    rx = math.sqrt(Ixxc/A)
    ry = math.sqrt(Iyyc/A)
    J = ((D-tw/2)+(W-tf/2))*tf**3/3
    Cwp = (tw**3*(D-tw/2)**3+tf**3*(W-tf/2)**3)/36                                  # 추가 검증 필요
    Iu = (Ixxc+Iyyc)/2+(((Ixxc-Iyyc)*(Ixxc-Iyyc)/4 + Ixyc**2)**0.5)
    Iv = (Ixxc+Iyyc)/2-(((Ixxc-Iyyc)*(Ixxc-Iyyc)/4 + Ixyc**2)**0.5)
    ru = math.sqrt(Iu/A)
    rv = math.sqrt(Iv/A)

    if Iyyc == Ixxc :
        theta = 45
    else : 
        theta = (math.atan((2*Ixyc)/(Iyyc-Ixxc)))/2*180/math.pi
        
    x_duplus = -(xc)
    y_duplus = (D-yc)
    x_duminus = (W-xc)
    y_duminus = -(yc)
    x_dvplus = (W-xc)
    y_dvplus = -(yc-tf)
    x_dvminus = -(xc)
    y_dvminus = -(yc)
    du1 = abs(y_duplus*math.cos(theta*math.pi/180)-x_duplus*math.sin(theta*math.pi/180))
    du2 = abs(y_duminus*math.cos(theta*math.pi/180)-x_duminus*math.sin(theta*math.pi/180))
    dv1 = abs(x_dvplus*math.cos(theta*math.pi/180)+y_dvplus*math.sin(theta*math.pi/180))
    dv2 = abs(x_dvminus*math.cos(theta*math.pi/180)+y_dvminus*math.sin(theta*math.pi/180))
    Sumin = min(Iu/du1, Iu/du2)
    Svmin = min(Iv/dv1, Iv/dv2)

    SectionPropList.append(A)
    SectionPropList.append(Sxx)
    SectionPropList.append(Syy)
    SectionPropList.append(Zxx)
    SectionPropList.append(Zyy)
    SectionPropList.append(Ixxc)
    SectionPropList.append(Iyyc)
    SectionPropList.append(Ixyc)
    SectionPropList.append(rx)
    SectionPropList.append(ry)
    SectionPropList.append(J)
    SectionPropList.append(Cwp)
    SectionPropList.append(theta)
    SectionPropList.append(xc)
    SectionPropList.append(yc)
    SectionPropList.append(Iu)
    SectionPropList.append(Iv)
    SectionPropList.append(Sumin)
    SectionPropList.append(Svmin)
    SectionPropList.append(ru)
    SectionPropList.append(rv)
    return SectionPropList

def T_Section_Prop_Calc(SectionPropList) :
    D = SectionPropList[2]
    W = SectionPropList[3]
    tw = SectionPropList[4]
    tf = SectionPropList[5]
    radi = SectionPropList[6]

    A = W*tf+D*tw-tw*tf
    xc = W/2
    yc = ((W*tf*(D-tf/2))+(((D-tf)*tw*(D-tf)/2)))/A
    Ixxc = ((D-tf)**3*tw/12)+((D-tf)*tw*((D-tf)/2-yc)**2)+(tf**3*W/12)+(tf*W*((D-tf/2)-yc)**2)
    Iyyc = ((D-tf)*tw**3+tf*W**3)/12
    Sxx = Ixxc/max(yc,D-yc)
    Syy = Iyyc/(W/2)
    if W*tf >= (D-tf)*tw :
        x = (tf*W+tw*tf-tw*D)/(2*W)
        Zxx = (W*(tf-x)*(tf-x)/2)+(W*x*x/2)+((D-tf)*tw*(x+(D-tf)/2))
    else : 
        x = (tw*D-tw*tf-tf*W)/(2*tw)
        Zxx = (W*tf*(tf/2+x))+(tw*x*x/2)+((D-tf-x)*tw*(D-tf-x)/2)
    Zyy = ((W/2)*tf*(W/4)+(D-tf)*tw/2*tw/4)*2
    rx = math.sqrt(Ixxc/A)
    ry = math.sqrt(Iyyc/A)
    J = (W*tf**3+(D-tf/2)*tw**3)/3
    Cwp = W**3*tf**3/144+(D-tf/2)**3*tw**3/36

    SectionPropList.append(A)
    SectionPropList.append(Sxx)
    SectionPropList.append(Syy)
    SectionPropList.append(Zxx)
    SectionPropList.append(Zyy)
    SectionPropList.append(Ixxc)
    SectionPropList.append(Iyyc)
    SectionPropList.append(rx)
    SectionPropList.append(ry)
    SectionPropList.append(J)
    SectionPropList.append(Cwp)
    SectionPropList.append(xc)
    SectionPropList.append(yc)
    return SectionPropList

def DoubleL_Section_Prop_Calc(SectionPropList) :
    D = SectionPropList[2]
    W = SectionPropList[3]
    tw = SectionPropList[4]
    tf = SectionPropList[5]
    radi = SectionPropList[6]
    Gap = SectionPropList[7]

    if D >= W :
        D = D
        W = W
    else : 
        D_temp = D
        tw_temp = tw
        W_temp = W
        tf_temp = tf
        SectionPropList[2] = W_temp
        SectionPropList[4] = tf_temp
        SectionPropList[3] = D_temp
        SectionPropList[5] = tw_temp
        D = W_temp
        W = D_temp
        tw = tf_temp
        tf = tw_temp

    A = 2*W*tf+2*D*tw-2*tw*tf
    xc = ((W*tf*W/2+tw*D*(W-tw/2)-tw*tf*(W-tw/2))+(W*tf*(W+Gap+W/2)+tw*D*(W+Gap+tw/2)-tw*tf*(W+Gap+tw/2)))/A
    yc = 2*(W*tf*tf/2+tw*D*D/2-tw*tf*tf/2)/A
    Ixxc = (((W-tw)*tf**3+tw*D**3)/12+(W-tw)*tf*(yc-tf/2)**2+tw*D*(yc-D/2)**2)*2
    Iyyc = ((tf*W**3+(D-tf)*tw**3)/12+tf*W*(xc-W/2)**2+(D-tf)*tw*(xc-(D-tw/2))**2)*2
    Sxx = Ixxc/(D-yc)
    Syy = Iyyc/(2*W+Gap-xc)
    Zxx = (tf*(D-(A/2/(2*W)))**2/2+W*(A/2/(2*W))**2/2+(W-tw)*((A/2/(2*W))-tw)**2/2)*2
    Zyy = (tf*W*(xc-W/2)+(D-tf)*tw*(xc-(D-tw/2)))*2
    rx = math.sqrt(Ixxc/A)
    ry = math.sqrt(Iyyc/A)
    J = 2*((D-tw/2)+(W-tf/2))*tf**3/3
    Cwp = 2*(tw**3*(D-tw/2)**3+tf**3*(W-tf/2)**3)/36                                    # 추가 검증 필요

    SectionPropList.append(A)
    SectionPropList.append(Sxx)
    SectionPropList.append(Syy)
    SectionPropList.append(Zxx)
    SectionPropList.append(Zyy)
    SectionPropList.append(Ixxc)
    SectionPropList.append(Iyyc)
    SectionPropList.append(rx)
    SectionPropList.append(ry)
    SectionPropList.append(J)
    SectionPropList.append(Cwp)
    SectionPropList.append(xc)
    SectionPropList.append(yc)

    return SectionPropList

In [5]:
def Section_Dict_Composing (SectionPropList) : 
    SectionNameKey = []
    Prop = []
    for innerlist in SectionPropList : 
        if innerlist[1] == "H" :
            DictKey = innerlist[0]
            DictVal = H_Section_Prop_Calc(innerlist)
            PropDict = dict(zip(HsectionKeys, DictVal))
            SectionNameKey.append(DictKey)
            Prop.append(PropDict)
        if innerlist[1] == "L" :
            DictKey = innerlist[0]
            DictVal = L_Section_Prop_Calc(innerlist)
            PropDict = dict(zip(LsectionKeys, DictVal))
            SectionNameKey.append(DictKey)
            Prop.append(PropDict)
        if innerlist[1] == "T" :
            DictKey = innerlist[0]
            DictVal = T_Section_Prop_Calc(innerlist)
            PropDict = dict(zip(TsectionKeys, DictVal))
            SectionNameKey.append(DictKey)
            Prop.append(PropDict)
        if innerlist[1] == "2L" :
            DictKey = innerlist[0]
            DictVal = DoubleL_Section_Prop_Calc(innerlist)
            PropDict = dict(zip(DoubleLsectionKeys, DictVal))
            SectionNameKey.append(DictKey)
            Prop.append(PropDict)
        if innerlist[1] == "2L" :
            DictKey = innerlist[0]+'_temp'
            DictVal = L_Section_Prop_Calc(innerlist[:7])
            PropDict = dict(zip(LsectionKeys, DictVal))
            SectionNameKey.append(DictKey)
            Prop.append(PropDict)
    SectionPropDict = dict(zip(SectionNameKey, Prop))
    return SectionPropDict

In [6]:
ColumnForceDict = Force_Dict_Composing(ColumnForceData, forceKeys)
GirderForceDict = Force_Dict_Composing(GirderForceData, forceKeys)
BeamForceDict = Force_Dict_Composing(BeamForceData, forceKeys)
ApplicationListDict = Section_Dict_Composing(Section)

ColumnSection = ApplicationListDict['2L-90x90x8']
GirderSection = ApplicationListDict['L-90x90x8']
BeamSection = ApplicationListDict['UB-203x102x23']

print(ApplicationListDict['2L-90x90x8'])
print(ApplicationListDict['2L-90x90x8_temp'])

{'SectionName': '2L-90x90x8', 'Shape': '2L', 'D': 90, 'W': 90, 'tw': 8, 'tf': 8, 'radi': 0, 'Gap': 10, 'Area': 2752, 'Sxx': 33046.29988590644, 'Syy': 49318.62456140351, 'Zxx': 59529.24444444445, 'Zyy': 83808.0, 'Ixx': 2133023.379844961, 'Iyy': 4685269.333333333, 'rx': 27.840279632248716, 'ry': 41.26131510302365, 'J': 58709.333333333336, 'Cwp': 36184519.11111111, 'xc': 95.0, 'yc': 25.453488372093023}
{'SectionName': '2L-90x90x8', 'Shape': '2L', 'D': 90, 'W': 90, 'tw': 8, 'tf': 8, 'radi': 0, 'Area': 1376, 'Sxx': 16523.14994295322, 'Syy': 16523.14994295322, 'Zxx': 29764.622222222224, 'Zyy': 29764.622222222224, 'Ixx': 1066511.6899224806, 'Iyy': 1066511.6899224806, 'Ixy': -633306.976744186, 'rx': 27.840279632248716, 'ry': 27.840279632248716, 'J': 29354.666666666668, 'Cwp': 18092259.555555556, 'theta': 45, 'xc': 25.453488372093023, 'yc': 25.453488372093023, 'Iu': 1699818.6666666665, 'Iv': 433204.7131782946, 'Sumin': 26710.07346638834, 'Svmin': 12034.577966381774, 'ru': 35.14730904825195, 'rv

In [7]:
def Calculate_Slenderness_Ratio (SectionPropDict,UnbracedSegment,ElemLength) : 
    rx = SectionPropDict['rx']
    ry = SectionPropDict['ry']
    if SectionPropDict['Shape'] == 'H' :
        Ly_ = ElemLength/UnbracedSegment
        SR = max(K*ElemLength/rx, K*Ly_/ry)
    if SectionPropDict['Shape'] == 'T':
        Ly_ = ElemLength/UnbracedSegment
        SR = max(K*ElemLength/rx, K*Ly_/ry)
    if SectionPropDict['Shape'] == 'L' :
        Ly_ = ElemLength/UnbracedSegment
        SR1 = max(K*ElemLength/rx, K*Ly_/ry)
        if SR1 <= 75 :
            SR = 60+0.8*SR1
        else : 
            SR = 45+SR1
    if SectionPropDict['Shape'] == '2L':
        Ly_ = ElemLength/UnbracedSegment
        SR = max(K*ElemLength/rx, K*Ly_/ry)
    return SR

In [8]:
def Calculate_Tensile_Strength (SectionPropDict) : 
    PhiP = 0.9
    Ag = SectionPropDict['Area']
    Pn = Fy*Ag

    ΦPn = PhiP*Pn

    return ΦPn 

In [9]:
def Calculate_Flexural_Buckling(SectionPropDict,UnbracedSegment,ElemLength) : 
    PhiP = 0.9
    if SectionPropDict['Shape'] == 'H' :
        Pn = Flexural_Buckling_HSection(SectionPropDict,UnbracedSegment,ElemLength)
    if SectionPropDict['Shape'] == 'L' :
        Pn = Flexural_Buckling_LSection(SectionPropDict,UnbracedSegment,ElemLength)
    if SectionPropDict['Shape'] == 'T' :
        Pn = Flexural_Buckling_TSection(SectionPropDict,UnbracedSegment,ElemLength)
    if SectionPropDict['Shape'] == '2L' :
        Pn = Flexural_Buckling_TSection(SectionPropDict,UnbracedSegment,ElemLength)
    ΦPn = PhiP*Pn
    return ΦPn

def Flexural_Buckling_HSection(SectionPropDict,UnbracedSegment,ElemLength) : 
    Ag = SectionPropDict['Area']
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tw = SectionPropDict['tw']
    tf = SectionPropDict['tf']
    radi = SectionPropDict['radi']
    SR = Calculate_Slenderness_Ratio(SectionPropDict,UnbracedSegment,ElemLength)
    Fe = (math.pi**2)*E/(SR**2)

    λrf = 0.56*math.sqrt(E/Fy)
    λrw = 1.49*math.sqrt(E/Fy)
    λf = W/(2*tf)
    λw = (D-2*tf-2*radi)/tw
    
    if SR <= 4.71*math.sqrt(E/(Fy)) : 
        Fn = 0.658**(Fy/Fe)*Fy
    else : 
        Fn = 0.877*Fe


    if (λrf > λf) and (λrw > λw) :      # Flexural Buckling of Member without Slender Element
        Pn = Fn*Ag

    elif (λrf < λf) and (λrw > λw) :    # Flexural Buckling of Member with Slender Flange
        Fel = (1.49*(λrf/λf))**2*Fy
        if λf <= λrf*math.sqrt(Fy/Fn) : 
            Ae = Ag
        else : 
            be = (W/2)*(1-0.22*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
            Ae = Ag-((W-2*be)*tf)*2
        
        Pn = Fn*Ae

    elif (λrf > λf) and (λrw < λw) :    # Flexural Buckling of Member with Slender Web
        Fel = (1.31*(λrw/λw))**2*Fy
        if λw <= λrw*math.sqrt(Fy/Fn) : 
            Ae = Ag
        else : 
            be = (D-2*tf-2*radi)*(1-0.18*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
            Ae = Ag-(((D-2*tf-2*radi)-be)*tw)

        Pn = Fn*Ae

    else :                              # Flexural Buckling of Member with Slender Flange & Web
        Felf = (1.49*(λrf/λf))**2*Fy
        Felw = (1.31*(λrw/λw))**2*Fy
        if λf <= λrf*math.sqrt(Fy/Fn) and λw <= λrw*math.sqrt(Fy/Fn) : 
            Ae = Ag
        elif λf > λrf*math.sqrt(Fy/Fn) and λw <= λrw*math.sqrt(Fy/Fn) : 
            be = (W/2)*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
            Ae = Ag-((W-2*be)*tf)*2
        elif λf <= λrf*math.sqrt(Fy/Fn) and λw > λrw*math.sqrt(Fy/Fn) : 
            be = (D-2*tf-2*radi)*(1-0.18*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
            Ae = Ag-(((D-2*tf-2*radi)-be)*tw)
        else : 
            be = (W/2)*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
            he = (D-2*tf-2*radi)*(1-0.18*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
            Ae = (be*2*tf)*2+(he*tw)

        Pn = Fn*Ae

    return Pn

def Flexural_Buckling_LSection(SectionPropDict,UnbracedSegment,ElemLength) :
    Ly_ = ElemLength/UnbracedSegment
    Ag = SectionPropDict['Area']
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tw = SectionPropDict['tw']
    tf = SectionPropDict['tf']
    SR = Calculate_Slenderness_Ratio(SectionPropDict,UnbracedSegment,ElemLength)
    SR1 = min(SR,200)
    Fe = (math.pi**2)*E/(SR1**2)

    λr = 0.45*math.sqrt(E/Fy)
    λf = W/tf
    λw = D/tw

    if SR1 <= 4.71*math.sqrt(E/(Fy)) : 
        Fn = 0.658**(Fy/Fe)*Fy
    else : 
        Fn = 0.877*Fe

    if (λr > λf) and (λr > λw) :      # Flexural Buckling of Member without Slender Element
        Pn = Fn*Ag

    elif (λr < λf) and (λr > λw) :    # Flexural Buckling of Member with Slender Flange
        Fel = (1.49*(λr/λf))**2*Fy
        if λf <= λr*math.sqrt(Fy/Fn) : 
            Ae = Ag
        else : 
            be = W*(1-0.22*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
            Ae = Ag-((W-be)*tf)
        
        Pn = Fn*Ae

    elif (λr > λf) and (λr < λw) :    # Flexural Buckling of Member with Slender Web
        Fel = (1.49*(λr/λw))**2*Fy
        if λw <= λr*math.sqrt(Fy/Fn) : 
            Ae = Ag
        else : 
            be = D*(1-0.22*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
            Ae = Ag-((D-be)*tw)

        Pn = Fn*Ae

    else :                              # Flexural Buckling of Member with Slender Flange & Web
        Felf = (1.49*(λr/λf))**2*Fy
        Felw = (1.49*(λr/λw))**2*Fy
        if λf <= λr*math.sqrt(Fy/Fn) and λw <= λr*math.sqrt(Fy/Fn) : 
            Ae = Ag
        elif λf > λr*math.sqrt(Fy/Fn) and λw <= λr*math.sqrt(Fy/Fn) : 
            be = W*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
            Ae = Ag-((W-be)*tf)
        elif λf <= λr*math.sqrt(Fy/Fn) and λw > λr*math.sqrt(Fy/Fn) : 
            be = D*(1-0.22*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
            Ae = Ag-((D-be)*tw)
        else : 
            be = W*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
            he = D*(1-0.22*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
            Ae = (be*tf)+(he*tw)

        Pn = Fn*Ae

    return Pn

def Flexural_Buckling_TSection(SectionPropDict,UnbracedSegment,ElemLength) : 
    Ag = SectionPropDict['Area']
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tw = SectionPropDict['tw']
    tf = SectionPropDict['tf']
    radi = SectionPropDict['radi']
    SR = Calculate_Slenderness_Ratio(SectionPropDict,UnbracedSegment,ElemLength)
    Fe = (math.pi**2)*E/(SR**2)

    λrf = 0.56*math.sqrt(E/Fy)
    λrw = 0.75*math.sqrt(E/Fy)
    λf = W/(2*tf)
    λw = D/tw
    
    if SR <= 4.71*math.sqrt(E/(Fy)) : 
        Fn = 0.658**(Fy/Fe)*Fy
    else : 
        Fn = 0.877*Fe


    if (λrf > λf) and (λrw > λw) :      # Flexural Buckling of Member without Slender Element
        Pn = Fn*Ag

    elif (λrf < λf) and (λrw > λw) :    # Flexural Buckling of Member with Slender Flange
        Fel = (1.49*(λrf/λf))**2*Fy
        if λf <= λrf*math.sqrt(Fy/Fn) : 
            Ae = Ag
        else : 
            be = (W/2)*(1-0.22*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
            Ae = Ag-((W-2*be)*tf)
        
        Pn = Fn*Ae

    elif (λrf > λf) and (λrw < λw) :    # Flexural Buckling of Member with Slender Web
        Fel = (1.49*(λrw/λw))**2*Fy
        if λw <= λrw*math.sqrt(Fy/Fn) : 
            Ae = Ag
        else : 
            be = (D-tf-2*radi)*(1-0.22*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
            Ae = Ag-(((D-tf-2*radi)-be)*tw)

        Pn = Fn*Ae

    else :                              # Flexural Buckling of Member with Slender Flange & Web
        Felf = (1.49*(λrf/λf))**2*Fy
        Felw = (1.49*(λrw/λw))**2*Fy
        if λf <= λrf*math.sqrt(Fy/Fn) and λw <= λrw*math.sqrt(Fy/Fn) : 
            Ae = Ag
        elif λf > λrf*math.sqrt(Fy/Fn) and λw <= λrw*math.sqrt(Fy/Fn) : 
            be = (W/2)*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
            Ae = Ag-((W-2*be)*tf)
        elif λf <= λrf*math.sqrt(Fy/Fn) and λw > λrw*math.sqrt(Fy/Fn) : 
            be = (D-2*tf-2*radi)*(1-0.22*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
            Ae = Ag-(((D-tf-2*radi)-be)*tw)
        else : 
            be = (W/2)*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
            he = (D-tf-2*radi)*(1-0.22*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
            Ae = (be*2*tf)+(he*tw)

        Pn = Fn*Ae

    return Pn

def Flexural_Buckling_DoubleLSection(SectionPropDict,UnbracedSegment,ElemLength) : 
    Ag = SectionPropDict['Area']
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tw = SectionPropDict['tw']
    tf = SectionPropDict['tf']
    radi = SectionPropDict['radi']
    SR = Calculate_Slenderness_Ratio(SectionPropDict,UnbracedSegment,ElemLength)
    Fe = (math.pi**2)*E/(SR**2)

    λrf = 0.45*math.sqrt(E/Fy)
    λrw = 0.45*math.sqrt(E/Fy)
    λf = W/tf
    λw = D/tw
    
    if SR <= 4.71*math.sqrt(E/(Fy)) : 
        Fn = 0.658**(Fy/Fe)*Fy
    else : 
        Fn = 0.877*Fe


    if (λrf > λf) and (λrw > λw) :      # Flexural Buckling of Member without Slender Element
        Pn = Fn*Ag

    elif (λrf < λf) and (λrw > λw) :    # Flexural Buckling of Member with Slender Flange
        Fel = (1.49*(λrf/λf))**2*Fy
        if λf <= λrf*math.sqrt(Fy/Fn) : 
            Ae = Ag
        else : 
            be = (W/2)*(1-0.22*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
            Ae = Ag-((W-2*be)*tf)
        
        Pn = Fn*Ae

    elif (λrf > λf) and (λrw < λw) :    # Flexural Buckling of Member with Slender Web
        Fel = (1.49*(λrw/λw))**2*Fy
        if λw <= λrw*math.sqrt(Fy/Fn) : 
            Ae = Ag
        else : 
            be = (D-tf-2*radi)*(1-0.22*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
            Ae = Ag-2*(((D-tf-2*radi)-be)*tw)

        Pn = Fn*Ae

    else :                              # Flexural Buckling of Member with Slender Flange & Web
        Felf = (1.49*(λrf/λf))**2*Fy
        Felw = (1.49*(λrw/λw))**2*Fy
        if λf <= λrf*math.sqrt(Fy/Fn) and λw <= λrw*math.sqrt(Fy/Fn) : 
            Ae = Ag
        elif λf > λrf*math.sqrt(Fy/Fn) and λw <= λrw*math.sqrt(Fy/Fn) : 
            be = (W/2)*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
            Ae = Ag-((W-2*be)*tf)
        elif λf <= λrf*math.sqrt(Fy/Fn) and λw > λrw*math.sqrt(Fy/Fn) : 
            be = (D-2*tf-2*radi)*(1-0.22*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
            Ae = Ag-2*(((D-tf-2*radi)-be)*tw)
        else : 
            be = (W/2)*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
            he = (D-tf-2*radi)*(1-0.22*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
            Ae = (be*2*tf)+2*(he*tw)

        Pn = Fn*Ae

    return Pn

In [10]:
def Calculate_Flexural_Torsional_Buckling(SectionPropDict,UnbracedSegment,ElemLength) :
    PhiP = 0.9
    if SectionPropDict['Shape'] == 'H' :
        Pn = Flexural_Torsional_Buckling_HSection(SectionPropDict,UnbracedSegment,ElemLength)
    if SectionPropDict['Shape'] == 'L' :
        Pn = Flexural_Torsional_Buckling_LSection(SectionPropDict,UnbracedSegment,ElemLength)
    if SectionPropDict['Shape'] == 'T' :
        Pn = Flexural_Torsional_Buckling_TSection(SectionPropDict,UnbracedSegment,ElemLength)
    if SectionPropDict['Shape'] == '2L' :
        Pn = Flexural_Torsional_Buckling_DoubleLSection(SectionPropDict,UnbracedSegment,ElemLength)
    ΦPn = PhiP*Pn
    return ΦPn
    
def Flexural_Torsional_Buckling_HSection(SectionPropDict,UnbracedSegment,ElemLength) : 
    Ly_ = ElemLength/UnbracedSegment
    Ixx = SectionPropDict['Ixx']
    Iyy = SectionPropDict['Iyy']
    Ag = SectionPropDict['Area']
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tw = SectionPropDict['tw']
    tf = SectionPropDict['tf']
    J = SectionPropDict['J']
    Cwp = SectionPropDict['Cwp']
    radi = SectionPropDict['radi']
    SR = Calculate_Slenderness_Ratio(SectionPropDict,UnbracedSegment,ElemLength)
    Fe = ((math.pi**2)*E*Cwp/((K*Ly_)**2)+77.2*J)/(Ixx+Iyy)

    λrf = 0.56*math.sqrt(E/Fy)
    λrw = 1.49*math.sqrt(E/Fy)
    λf = W/(2*tf)
    λw = (D-2*tf-2*radi)/tw
    
    if SR <= 4.71*math.sqrt(E/(Fy)) : 
        Fn = 0.658**(Fy/Fe)*Fy
    else : 
        Fn = 0.877*Fe


    if (λrf > λf) and (λrw > λw) :      # Flexural Buckling of Member without Slender Element
        Pn = Fn*Ag

    elif (λrf < λf) and (λrw > λw) :    # Flexural Buckling of Member with Slender Flange
        Fel = (1.49*(λrf/λf))**2*Fy
        if λf <= λrf*math.sqrt(Fy/Fn) : 
            Ae = Ag
        else : 
            be = (W/2)*(1-0.22*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
            Ae = Ag-((W-2*be)*tf)*2
        
        Pn = Fn*Ae

    elif (λrf > λf) and (λrw < λw) :    # Flexural Buckling of Member with Slender Web
        Fel = (1.31*(λrw/λw))**2*Fy
        if λw <= λrw*math.sqrt(Fy/Fn) : 
            Ae = Ag
        else : 
            be = (D-2*tf-2*radi)*(1-0.18*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
            Ae = Ag-(((D-2*tf-2*radi)-be)*tw)

        Pn = Fn*Ae

    else :                              # Flexural Buckling of Member with Slender Flange & Web
        Felf = (1.49*(λrf/λf))**2*Fy
        Felw = (1.31*(λrw/λw))**2*Fy
        if λf <= λrf*math.sqrt(Fy/Fn) and λw <= λrw*math.sqrt(Fy/Fn) : 
            Ae = Ag
        elif λf > λrf*math.sqrt(Fy/Fn) and λw <= λrw*math.sqrt(Fy/Fn) : 
            be = (W/2)*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
            Ae = Ag-((W-2*be)*tf)*2
        elif λf <= λrf*math.sqrt(Fy/Fn) and λw > λrw*math.sqrt(Fy/Fn) : 
            be = (D-2*tf-2*radi)*(1-0.18*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
            Ae = Ag-(((D-2*tf-2*radi)-be)*tw)
        else : 
            be = (W/2)*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
            he = (D-2*tf-2*radi)*(1-0.18*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
            Ae = (be*2*tf)*2+(he*tw)

        Pn = Fn*Ae

    return Pn
    
def Flexural_Torsional_Buckling_LSection(SectionPropDict,UnbracedSegment,ElemLength) : 
    x = Symbol('x')
    Ly_ = ElemLength/UnbracedSegment
    Ag = SectionPropDict['Area']
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tw = SectionPropDict['tw']
    tf = SectionPropDict['tf']
    Ixx = SectionPropDict['Ixx']
    Iyy = SectionPropDict['Iyy']
    rx = SectionPropDict['rx']
    ry = SectionPropDict['ry']
    J = SectionPropDict['J']
    Cwp = SectionPropDict['Cwp']
    xc = SectionPropDict['xc']
    yc = SectionPropDict['yc']
    SR = Calculate_Slenderness_Ratio(SectionPropDict,UnbracedSegment,ElemLength)
    SR1 = min(SR,200)

    if D/tw > 0.71*math.sqrt(E/Fy) :
        xo = tw/2-xc
        yo = tf/2-yc
        ro = math.sqrt(xo**2+yo**2+(Ixx+Iyy)/Ag)
        Fex = math.pi**2*E/(ElemLength/rx)
        Fey = math.pi**2*E/(Ly_/ry)
        Fez = ((math.pi**2)*E*Cwp/((Ly_)**2)+77.2*J)/(Ag*ro**2)

        Fe_Root = solve(((x-Fex)*(x-Fey)*(x-Fez))-(x**2*(x-Fey)*(xo/ro)**2)-(x**2*(x-Fex)*(yo/ro)**2),x)
        Real_Root = [sol.evalf() for sol in Fe_Root if sol.is_real]
        Fe = min(Real_Root)

        λr = 0.45*math.sqrt(E/Fy)
        λf = W/tf
        λw = D/tw
        
        if SR1 <= 4.71*math.sqrt(E/(Fy)) : 
            Fn = 0.658**(Fy/Fe)*Fy
        else : 
            Fn = 0.877*Fe

        if (λr > λf) and (λr > λw) :      # Flexural Buckling of Member without Slender Element
            Pn = Fn*Ag

        elif (λr < λf) and (λr > λw) :    # Flexural Buckling of Member with Slender Flange
            Fel = (1.49*(λr/λf))**2*Fy
            if λf <= λr*math.sqrt(Fy/Fn) : 
                Ae = Ag
            else : 
                be = W*(1-0.22*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
                Ae = Ag-((W-be)*tf)
            
            Pn = Fn*Ae

        elif (λr > λf) and (λr < λw) :    # Flexural Buckling of Member with Slender Web
            Fel = (1.49*(λr/λw))**2*Fy
            if λw <= λr*math.sqrt(Fy/Fn) : 
                Ae = Ag
            else : 
                be = D*(1-0.22*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
                Ae = Ag-((D-be)*tw)

            Pn = Fn*Ae

        else :                              # Flexural Buckling of Member with Slender Flange & Web
            Felf = (1.49*(λr/λf))**2*Fy
            Felw = (1.49*(λr/λw))**2*Fy
            if λf <= λr*math.sqrt(Fy/Fn) and λw <= λr*math.sqrt(Fy/Fn) : 
                Ae = Ag
            elif λf > λr*math.sqrt(Fy/Fn) and λw <= λr*math.sqrt(Fy/Fn) : 
                be = W*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
                Ae = Ag-((W-be)*tf)
            elif λf <= λr*math.sqrt(Fy/Fn) and λw > λr*math.sqrt(Fy/Fn) : 
                be = D*(1-0.22*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
                Ae = Ag-((D-be)*tw)
            else : 
                be = W*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
                he = D*(1-0.22*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
                Ae = (be*tf)+(he*tw)

            Pn = Fn*Ae

    else : 
        Pn = Fy*Ag

    return Pn

def Flexural_Torsional_Buckling_TSection(SectionPropDict,UnbracedSegment,ElemLength) : 
    Ly_ = ElemLength/UnbracedSegment
    Ag = SectionPropDict['Area']
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tw = SectionPropDict['tw']
    tf = SectionPropDict['tf']
    Ixx = SectionPropDict['Ixx']
    Iyy = SectionPropDict['Iyy']
    ry = SectionPropDict['ry']
    J = SectionPropDict['J']
    Cwp = SectionPropDict['Cwp']
    radi = SectionPropDict['radi']
    xc = SectionPropDict['xc']
    yc = SectionPropDict['yc']
    SR = Calculate_Slenderness_Ratio(SectionPropDict,UnbracedSegment,ElemLength)
    xo = W/2-xc
    yo = D-tf/2-yc
    ro = math.sqrt(xo**2+yo**2+(Ixx+Iyy)/Ag)
    H = 1-(xo**2+yo**2)/ro**2
    Fey = math.pi**2*E/(Ly_/ry)**2
    Fez = ((math.pi**2)*E*Cwp/((ElemLength)**2)+77.2*J)/(Ag*ro**2)
    Fe = (Fey+Fez)/(2*H)*(1-math.sqrt(1-4*Fey*Fez*H/((Fey+Fez)**2)))

    λrf = 0.56*math.sqrt(E/Fy)
    λrw = 0.75*math.sqrt(E/Fy)
    λf = W/(2*tf)
    λw = D/tw
    
    if SR <= 4.71*math.sqrt(E/(Fy)) : 
        Fn = 0.658**(Fy/Fe)*Fy
    else : 
        Fn = 0.877*Fe


    if (λrf > λf) and (λrw > λw) :      # Flexural Buckling of Member without Slender Element
        Pn = Fn*Ag

    elif (λrf < λf) and (λrw > λw) :    # Flexural Buckling of Member with Slender Flange
        Fel = (1.49*(λrf/λf))**2*Fy
        if λf <= λrf*math.sqrt(Fy/Fn) : 
            Ae = Ag
        else : 
            be = (W/2)*(1-0.22*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
            Ae = Ag-((W-2*be)*tf)
        
        Pn = Fn*Ae

    elif (λrf > λf) and (λrw < λw) :    # Flexural Buckling of Member with Slender Web
        Fel = (1.49*(λrw/λw))**2*Fy
        if λw <= λrw*math.sqrt(Fy/Fn) : 
            Ae = Ag
        else : 
            be = (D-tf-2*radi)*(1-0.22*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
            Ae = Ag-(((D-tf-2*radi)-be)*tw)

        Pn = Fn*Ae

    else :                              # Flexural Buckling of Member with Slender Flange & Web
        Felf = (1.49*(λrf/λf))**2*Fy
        Felw = (1.49*(λrw/λw))**2*Fy
        if λf <= λrf*math.sqrt(Fy/Fn) and λw <= λrw*math.sqrt(Fy/Fn) : 
            Ae = Ag
        elif λf > λrf*math.sqrt(Fy/Fn) and λw <= λrw*math.sqrt(Fy/Fn) : 
            be = (W/2)*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
            Ae = Ag-((W-2*be)*tf)
        elif λf <= λrf*math.sqrt(Fy/Fn) and λw > λrw*math.sqrt(Fy/Fn) : 
            be = (D-2*tf-2*radi)*(1-0.22*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
            Ae = Ag-(((D-tf-2*radi)-be)*tw)
        else : 
            be = (W/2)*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
            he = (D-tf-2*radi)*(1-0.22*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
            Ae = (be*2*tf)+(he*tw)

        Pn = Fn*Ae

    return Pn

def Flexural_Torsional_Buckling_DoubleLSection(SectionPropDict,UnbracedSegment,ElemLength) : 
    Ly_ = ElemLength/UnbracedSegment
    Ag = SectionPropDict['Area']
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tw = SectionPropDict['tw']
    tf = SectionPropDict['tf']
    Ixx = SectionPropDict['Ixx']
    Iyy = SectionPropDict['Iyy']
    ry = SectionPropDict['ry']
    J = SectionPropDict['J']
    Cwp = SectionPropDict['Cwp']
    radi = SectionPropDict['radi']
    xc = SectionPropDict['xc']
    yc = SectionPropDict['yc']
    SR = Calculate_Slenderness_Ratio(SectionPropDict,UnbracedSegment,ElemLength)
    xo = xc
    yo = D-tf/2-yc
    ro = math.sqrt(xo**2+yo**2+(Ixx+Iyy)/Ag)
    H = 1-(xo**2+yo**2)/ro**2
    Fey = math.pi**2*E/(Ly_/ry)**2
    Fez = ((math.pi**2)*E*Cwp/((ElemLength)**2)+77.2*J)/(Ag*ro**2)
    Fe = (Fey+Fez)/(2*H)*(1-math.sqrt(1-4*Fey*Fez*H/((Fey+Fez)**2)))

    λrf = 0.45*math.sqrt(E/Fy)
    λrw = 0.45*math.sqrt(E/Fy)
    λf = W/(2*tf)
    λw = D/tw
    
    if SR <= 4.71*math.sqrt(E/(Fy)) : 
        Fn = 0.658**(Fy/Fe)*Fy
    else : 
        Fn = 0.877*Fe


    if (λrf > λf) and (λrw > λw) :      # Flexural Buckling of Member without Slender Element
        Pn = Fn*Ag

    elif (λrf < λf) and (λrw > λw) :    # Flexural Buckling of Member with Slender Flange
        Fel = (1.49*(λrf/λf))**2*Fy
        if λf <= λrf*math.sqrt(Fy/Fn) : 
            Ae = Ag
        else : 
            be = (W/2)*(1-0.22*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
            Ae = Ag-((W-2*be)*tf)
        
        Pn = Fn*Ae

    elif (λrf > λf) and (λrw < λw) :    # Flexural Buckling of Member with Slender Web
        Fel = (1.49*(λrw/λw))**2*Fy
        if λw <= λrw*math.sqrt(Fy/Fn) : 
            Ae = Ag
        else : 
            be = (D-tf-2*radi)*(1-0.22*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
            Ae = Ag-2*(((D-tf-2*radi)-be)*tw)

        Pn = Fn*Ae

    else :                              # Flexural Buckling of Member with Slender Flange & Web
        Felf = (1.49*(λrf/λf))**2*Fy
        Felw = (1.49*(λrw/λw))**2*Fy
        if λf <= λrf*math.sqrt(Fy/Fn) and λw <= λrw*math.sqrt(Fy/Fn) : 
            Ae = Ag
        elif λf > λrf*math.sqrt(Fy/Fn) and λw <= λrw*math.sqrt(Fy/Fn) : 
            be = (W/2)*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
            Ae = Ag-((W-2*be)*tf)
        elif λf <= λrf*math.sqrt(Fy/Fn) and λw > λrw*math.sqrt(Fy/Fn) : 
            be = (D-2*tf-2*radi)*(1-0.22*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
            Ae = Ag-2*(((D-tf-2*radi)-be)*tw)
        else : 
            be = (W/2)*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
            he = (D-tf-2*radi)*(1-0.22*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
            Ae = (be*2*tf)+2*(he*tw)

        Pn = Fn*Ae

    return Pn

In [11]:
def Calculate_Lateral_Torsional_Buckling(SectionPropDict,UnbracedSegment,ElemLength) : 
    PhiM = 0.9
    if SectionPropDict['Shape'] == 'H' :
        Mnx = Lateral_Torsional_Buckling_HSection(SectionPropDict,UnbracedSegment,ElemLength)
    if SectionPropDict['Shape'] == 'L' :
        Mnx = Lateral_Torsional_Buckling_LSection(SectionPropDict,UnbracedSegment,ElemLength)
    if SectionPropDict['Shape'] == 'T' :
        Mnx = Lateral_Torsional_Buckling_TSection(SectionPropDict,UnbracedSegment,ElemLength)
    if SectionPropDict['Shape'] == '2L':
        Mnx = Lateral_Torsional_Buckling_DoubleLSection(SectionPropDict,UnbracedSegment,ElemLength)
    ΦMnx = PhiM*Mnx
    return ΦMnx

def Lateral_Torsional_Buckling_HSection(SectionPropDict,UnbracedSegment,ElemLength) :
    Sxx = SectionPropDict['Sxx']
    Zxx = SectionPropDict['Zxx']
    Iyy = SectionPropDict['Iyy']
    ry = SectionPropDict['ry']
    J = SectionPropDict['J']
    Cwp = SectionPropDict['Cwp']
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tw = SectionPropDict['tw']
    tf = SectionPropDict['tf']
    radi = SectionPropDict['radi']
    Mp = Fy*Zxx
    Cb = 1.0

    λw = (D-2*tf-2*radi)/tw
    λpw = 3.76*math.sqrt(E/Fy)
    λrw = 5.70*math.sqrt(E/Fy)

    if UnbracedSegment == 0 :
        Ly_ = 0
    else : 
        Ly_ = ElemLength/UnbracedSegment

    if λw <= λpw :                                       # Compact Web & Compact/Non-Compact/Slender Flange
        Lb = Ly_
        rts = math.sqrt(math.sqrt(Iyy*Cwp)/Sxx)
        Lp = 1.76*ry*math.sqrt(E/Fy)
        Lr = 1.95*rts*E/(0.7*Fy)*math.sqrt(J/(Sxx*(D-tf))+math.sqrt((J/(Sxx*(D-tf)))**2+6.76*(0.7*Fy/E)**2))
        Fcr = Cb*math.pi**2*E/(Lb/rts)**2*math.sqrt(1+0.078*J/(Sxx*(D-tf))*(Lb/rts)**2)

        if Lb <= Lp : 
            Mnx = Mp
        elif Lp < Lb <= Lr : 
            Mnx = min(Cb*(Mp-(Mp-0.7*Fy*Sxx)*(Lb-Lp)/(Lr-Lp)),Mp)
        elif Lb > Lr : 
            Mnx = min(Fcr*Sxx,Mp)

    elif λpw < λw <= λrw :                                # Non-Compact Web & Compact/Non-Compact/Slender Flange
        Lb = Ly_
        Iyc = (tf*W**3)/12
        Sxc = Sxx
        Mxc = Fy * Sxc
        if Iyc/Iyy > 0.23 : 
            if λw <= λpw : 
                Rpc = Mp/Mxc
                J_ = J
            else : 
                Rpc = min((Mp/Mxc-(Mp/Mxc-1)*(λw-λpw)/(λrw-λpw)),Mp-Mxc)
                J_ = J
        else : 
            Rpc = 1
            J_ = 0

        FL = 0.7*Fy
        aw = ((D-2*tf-2*radi)*tw)/(W*tf)
        rt = W/math.sqrt(12*(1+1/6*aw))
        Lp = 1.1*rt*math.sqrt(E/Fy)
        Lr = 1.95*rt*(E/FL)*math.sqrt(J_/(Sxx*(D-tf))+math.sqrt((J_/(Sxx*(D-tf)))**2+6.76*(0.7*FL/E)**2))

        Fcr = Cb*math.pi**2*E/(Lb/rt)**2*math.sqrt(1+0.078*J_/(Sxx*(D-tf))*(Lb/rt)**2)

        if Lb <= Lp : 
            Mnx = Rpc*Mxc
        elif Lp < Lb <= Lr : 
            Mnx = min(Cb*(Rpc*Mxc-(Rpc*Mxc-0.7*FL*Sxx)*(Lb-Lp)/(Lr-Lp)),Rpc*Mxc)
        elif Lb > Lr : 
            Mnx = min(Fcr*Sxx,Rpc*Mxc)

    else :                                                 # Slender Web & Compact/Non-Compact/Slender Flange
        Lb = Ly_
        Sxc = Sxx
        aw = ((D-2*tf-2*radi)*tw)/(W*tf)
        rt = W/math.sqrt(12*(1+1/6*aw))
        Lp = 1.1*rt*math.sqrt(E/Fy)
        Lr = math.pi*rt*math.sqrt(E/(0.7*Fy))
        Rpg = min(1-(aw/(1200+300*aw))*((D-2*tf-2*radi)/tw-5.7*math.sqrt(E/Fy)),1)

        if Lb <= Lp : 
            Mnx = Rpg*Fy*Sxc
        elif Lp < Lb <= Lr : 
            Fcr = Cb*(Fy-(0.3*Fy)*(Lb-Lp)/(Lr-Lp))
            Mnx = min(Rpg*Fcr*Sxc,Fy)
        elif Lb > Lr : 
            Fcr = Cb*math.pi**2*E/(Lb/((W/2)/math.sqrt(12*(1+1/6*(((D-2*tf-2*radi)*tw)/((W/2)*tf))))))**2
            Mnx = min(Rpg*Fcr*Sxc,Fy)
    return Mnx

def Lateral_Torsional_Buckling_LSection(SectionPropDict,UnbracedSegment,ElemLength) :                                           # Equal Angle만 적용
    A = SectionPropDict['Area']
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tw = SectionPropDict['tw']
    tf = SectionPropDict['tf']
    theta = SectionPropDict['theta']
    xc = SectionPropDict['xc']
    yc = SectionPropDict['yc']
    Iu = SectionPropDict['Iu']
    Sumin = SectionPropDict['Sumin']
    ru = SectionPropDict['ru']
    rv = SectionPropDict['rv']

    if UnbracedSegment == 0 :
        Ly_ = 0
    else : 
        Ly_ = ElemLength/UnbracedSegment
        
    Lb = Ly_

    if D == W and tw == tf :
        βw = 0
    else : 
        x0 = xc-tw/2
        y0 = yc-tf/2
        u0 = y0*math.sin(theta*math.pi/180)+x0*math.cos(theta*math.pi/180)
        v0 = y0*math.cos(theta*math.pi/180)-x0*math.sin(theta*math.pi/180)
        C1 = ((x0**2)/2)*((y0**2)-(y0-(D-tf/2))**2)+(((y0**4)-(y0-(D-tf/2))**4)/4)+(y0/3)*((x0**3)-(x0-(W-tw/2))**3)+(y0**3)*(W-tw/2)
        C2 = (x0/3)*((y0**3)-(y0-(D-tf/2))**3)+((x0**3)*(y0-(D-tf/2)))+(((x0**4)-(x0-(W-tw/2))**4)/4)+((y0**2)/2)*((x0**2)-(x0-(W-tw/2))**2)
        βw = tf*(C1*math.cos(theta*math.pi/180)-C2*math.sin(theta*math.pi/180))/Iu-2*v0     # 추가 확인 필요

    Myu = Sumin*Fy
    Cb = 1
    rz = min(ru,rv)

    Mcr = (9*E*A*rz*tf*Cb)/(8*Lb)*(math.sqrt(1+(4.4*βw*rz/(Lb*tf))**2)+(4.4*βw*rz/(Lb*tf)))

    if Myu/Mcr <= 1.0 :
        Mnx = min((1.92-1.17*math.sqrt(Myu/Mcr))*Myu, 1.5*Myu)
    else : 
        Mnx = (0.92-0.17*Mcr/Myu)*Mcr
        
    return Mnx

def Lateral_Torsional_Buckling_TSection(SectionPropDict,UnbracedSegment,ElemLength) :
    Sxx = SectionPropDict['Sxx']
    Zxx = SectionPropDict['Zxx']
    Iyy = SectionPropDict['Iyy']
    ry = SectionPropDict['ry']
    J = SectionPropDict['J']
    D = SectionPropDict['D']

    if UnbracedSegment == 0 :
        Ly_ = 0
    else : 
        Ly_ = ElemLength/UnbracedSegment

    Lb = Ly_

    Lp = 1.76*ry*math.sqrt(E/Fy)
    Lr = 1.95*(E/Fy)*math.sqrt(Iyy*J)/Sxx*math.sqrt(2.36*(Fy/E)*D*Sxx/J+1)
    My = Fy*Sxx
    Mp1 = min(Fy*Zxx, 1.6*My)                                   # Tee stem in Tension
    B1 = 2.3*(D/Lb)*math.sqrt(Iyy/J)
    Mcr1 = 1.95*E/Lb*math.sqrt(Iyy*J)*(B1+math.sqrt(1+B1**2))
    Mp2 = My                                                    # Tee stem in Compression
    B2 = -2.3*(D/Lb)*math.sqrt(Iyy/J)
    Mcr2 = 1.95*E/Lb*math.sqrt(Iyy*J)*(B2+math.sqrt(1+B2**2))

    if Lb <= Lp :                                               # Stem in tension
        Mnx1 = Mp1
    if Lp < Lb <= Lr :
        Mnx1 = Mp1-(Mp1-My)*(Lb-Lp)/(Lr-Lp)
    else : 
        Mnx1 = Mcr1

    Mnx2 = min(Mcr2,Mp2)                                          # Stem in compression

    Mnx = min(Mnx1,Mnx2)

    return Mnx

def Lateral_Torsional_Buckling_DoubleLSection(SectionPropDict,UnbracedSegment,ElemLength) :
    Sxx = SectionPropDict['Sxx']
    Zxx = SectionPropDict['Zxx']
    Iyy = SectionPropDict['Iyy']
    ry = SectionPropDict['ry']
    J = SectionPropDict['J']
    D = SectionPropDict['D']
    
    if UnbracedSegment == 0 :
        Ly_ = 0
    else : 
        Ly_ = ElemLength/UnbracedSegment

    Lb = Ly_

    Lp = 1.76*ry*math.sqrt(E/Fy)
    Lr = 1.95*(E/Fy)*math.sqrt(Iyy*J)/Sxx*math.sqrt(2.36*(Fy/E)*D*Sxx/J+1)
    My = Fy*Sxx
    Mp1 = min(Fy*Zxx, 1.6*My)
    B1 = 2.3*(D/Lb)*math.sqrt(Iyy/J)
    Mcr1 = 1.95*E/Lb*math.sqrt(Iyy*J)*(B1+math.sqrt(1+B1**2))
    B2 = -2.3*(D/Lb)*math.sqrt(Iyy/J)
    Mcr2 = 1.95*E/Lb*math.sqrt(Iyy*J)*(B2+math.sqrt(1+B2**2))

    if Lb <= Lp :                                               # Stem in tension
        Mnx1 = Mp1
    if Lp < Lb <= Lr :
        Mnx1 = Mp1-(Mp1-My)*(Lb-Lp)/(Lr-Lp)
    else : 
        Mnx1 = Mcr1

    if My/Mcr2 <= 1.0 : 
        Mnx2 = min((1.92-1.17*math.sqrt(My/Mcr2))*My, 1.5*My)
    else : 
        Mnx2 = (0.92-0.17*Mcr2/My)*Mcr2

    Mnx = min(Mnx1,Mnx2)

    return Mnx

In [12]:
def Calculate_Local_Buckling(SectionPropDict) : 
    PhiM = 0.9
    if SectionPropDict['Shape'] == 'H' :
        Mnx = Compression_Flange_Local_Buckling_Hsection(SectionPropDict)
    if SectionPropDict['Shape'] == 'L' :
        Mnx = Major_Axis_Leg_Local_Buckling_Lsection(SectionPropDict)
    if SectionPropDict['Shape'] == 'T' :
        Mnx = Local_Buckling_Tsection(SectionPropDict)
    if SectionPropDict['Shape'] == '2L':
        Mnx = Local_Buckling_DoubleLsection(SectionPropDict)
    ΦMnx = PhiM*Mnx
    return ΦMnx

def Compression_Flange_Local_Buckling_Hsection(SectionPropDict) :
    Sxx = SectionPropDict['Sxx']
    Zxx = SectionPropDict['Zxx']
    Iyy = SectionPropDict['Iyy']
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tw = SectionPropDict['tw']
    tf = SectionPropDict['tf']
    radi = SectionPropDict['radi']
    Mp = Fy * Zxx

    λf = W/(2*tf)
    λw = (D-2*tf-2*radi)/tw
    λpf = 0.38*math.sqrt(E/Fy)
    λpw = 3.76*math.sqrt(E/Fy)
    λrf = 1.00*math.sqrt(E/Fy)
    λrw = 5.70*math.sqrt(E/Fy)

    if λw <= λpw :
        if λf <= λpf :                                                   # Compact Web & Compact Flange
            Mnx = Mp
        elif λpf < λf <= λrf :                                           # Compact Web & Non-Compact Flange
            Mnx = Mp-(Mp-0.7*Fy*Sxx)*(λf-λpf)/(λrf-λpf)
        elif λrf < λf :                                                  # Compact Web & Slender Flange
            kc = min(max(4/(math.sqrt((D-2*tf-2*radi)/tw)),0.35),0.76)
            Mnx = 0.9*E*kc*Sxx/λf**2


    elif λpw < λw <= λrw :
        Iyc = (tf*D**3)/12
        Sxc = Sxx
        Mxc = Fy * Sxc
        FL = 0.7*Fy
        if Iyc/Iyy > 0.23 : 
            if λw <= λpw : 
                Rpc = Mp/Mxc
            else : 
                Rpc = min((Mp/Mxc-(Mp/Mxc-1)*(λw-λpw)/(λrw-λpw)),Mp-Mxc)
        else : 
            Rpc = 1

        if λf <= λpf :                                                   # Non-Compact Web & Compact Flange
            Mnx = Rpc*Mxc
        elif λpf < λf <= λrf :                                           # Non-Compact Web & Non-Compact Flange
            Mnx = Rpc*Mxc-(Rpc*Mxc-FL*Sxc)*(λf-λpf)/(λrf-λpf)
        elif λrf < λf :                                                  # Non-Compact Web & Slender Flange
            kc = min(max(4/(math.sqrt((D-2*tf-2*radi)/tw)),0.35),0.76)
            Mnx = 0.9*E*kc*Sxc/λf**2

    elif λrw < λw : 
        Sxc = Sxx
        aw = ((D-2*tf-2*radi)*tw)/((W/2)*tf)
        Rpg = min(1-(aw/(1200+300*aw))*((D-2*tf-2*radi)/tw-5.7*math.sqrt(E/Fy)),1)

        if λf <= λpf :                                                   # Slender Web & Compact Flange
            Mnx = Rpg*Fy*Sxc
        elif λpf < λf <= λrf :                                           # Slender Web & Non-Compact Flange
            Fcr = Fy-(0.3*Fy)*(λf-λpf)/(λrf-λpf)
            Mnx = Rpg*Fcr*Sxc
        elif λrf < λf :                                                  # Slender Web & Slender Flange
            kc = min(max(4/(math.sqrt((D-2*tf-2*radi)/tw)),0.35),0.76)
            Fcr = 0.9*E*kc/λf**2
            Mnx = Rpg*Fcr*Sxc

    return Mnx

def Major_Axis_Leg_Local_Buckling_Lsection(SectionPropDict) :
    W = SectionPropDict['W']
    tf = SectionPropDict['tf']
    Sumin = SectionPropDict['Sumin']

    λf = W/tf
    λp = 0.54*math.sqrt(E/Fy)
    λr = 0.91*math.sqrt(E/Fy)

    if λf <= λp : 
        Mnu = Sumin*Fy
        Mnx = 1.5*Mnu
    if λp < λf <= λr : 
        Mnx = Fy*Sumin*(2.43-1.72*(λf)*math.sqrt(Fy/E))
    if λr < λf : 
        Fcr = 0.71*E/λf**2
        Mnx = Fcr*Sumin

    return Mnx

def Local_Buckling_Tsection(SectionPropDict) :
    Sxx = SectionPropDict['Sxx']
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tf = SectionPropDict['tf']
    tw = SectionPropDict['tw']

    My = Fy*Sxx
    Mp = My
    λf = W/(2*tf)
    λw = D/tw
    λpf = 0.38*math.sqrt(E/Fy)
    λrf = 1.00*math.sqrt(E/Fy)
    λpw = 0.84*math.sqrt(E/Fy)
    λrw = 1.52*math.sqrt(E/Fy)

    if λf <= λpf :                                                  # Compression Flange Local Buckling
        Mnx1 = My
    if λpf < λf <= λrf : 
        Sxc = Sxx
        Mnx1 = min((Mp-(Mp-0.7*Fy*Sxc)*(λf/λpf)/(λrf-λpf)),1.6*My)
    if λrf < λf : 
        Sxc = Sxx
        Mnx1 = 0.7*E*Sxc/(λf)**2

    if λw <= λpw :                                                  # Tee Stem Local Buckling
        Fcr = Fy
    if λpw < λw <= λrw : 
        Fcr = (1.43-0.515*λw*math.sqrt(Fy/E))*Fy
    if λrw < λw :
        Fcr = 1.52*E/λw**2

    Mnx2 = Fcr*Sxx

    Mnx = min(Mnx1,Mnx2)
    
    return Mnx

def Local_Buckling_DoubleLsection(SectionPropDict) :
    Sxx = SectionPropDict['Sxx']
    Ixx = SectionPropDict['Ixx']
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tf = SectionPropDict['tf']
    tw = SectionPropDict['tw']
    yc = SectionPropDict['yc']

    My = Fy*Sxx
    Mp = 1.5*My
    λf = W/tf
    λw = D/tw
    λpf = 0.54*math.sqrt(E/Fy)
    λrf = 0.91*math.sqrt(E/Fy)
    λpw = 0.54*math.sqrt(E/Fy)
    λrw = 0.91*math.sqrt(E/Fy)
    Sxc = Ixx/yc

    if λf <= λpf :                                                  # Compression Flange Local Buckling
        Mnx1 = Mp
    if λpf < λf <= λrf : 
        Mnx1 = Fy*Sxc*(2.43-1.72*(λf)*math.sqrt(Fy/E))
    if λrf < λf : 
        Fcr = 0.71*E/λf**2
        Mnx1 = Fcr*Sxc

    if λw <= λpw :                                                  # Web Local Buckling
        Mnx2 = Mp
    if λpw < λw <= λrw : 
        Mnx2 = Fy*Sxc*(2.43-1.72*(λw)*math.sqrt(Fy/E))
    if λrw < λw : 
        Fcr = 0.71*E/λw**2
        Mnx2 = Fcr*Sxc

    Mnx = min(Mnx1,Mnx2)
    
    return Mnx


In [13]:
def Calculate_Tension_Element_Yielding(SectionPropDict) : 
    PhiM = 0.9
    if SectionPropDict['Shape'] == 'H' :
        Mnx = Tension_Flange_Yielding_HSection(SectionPropDict)
    if SectionPropDict['Shape'] == 'L' :
        Mnx = Tension_Leg_Yielding_LSection(SectionPropDict)
    if SectionPropDict['Shape'] == 'T' :
        Mnx = Tension_Tee_Stem_Yielding_TSection(SectionPropDict)
    if SectionPropDict['Shape'] == '2L' :
        Mnx = Tension_Tee_Stem_Yielding_TSection(SectionPropDict)
    ΦMnx = PhiM*Mnx
    return ΦMnx

def Tension_Flange_Yielding_HSection(SectionPropDict) : 
    Zxx = SectionPropDict['Zxx']
    Mp = Fy*Zxx

    Mnx = Mp

    return Mnx

def Tension_Leg_Yielding_LSection(SectionPropDict) : 
    Sumin = SectionPropDict['Sumin']

    Mnu = Sumin*Fy
    Mnx = 1.5*Mnu

    return Mnx

def Tension_Tee_Stem_Yielding_TSection(SectionPropDict) : 
    Sxx = SectionPropDict['Sxx']
    Zxx = SectionPropDict['Zxx']

    My = Fy*Sxx
    Mp = min(Fy*Zxx,1.6*My)

    Mnx = Mp

    return Mnx

In [14]:
def Calculate_Minor_Axis_Flange_Local_Buckling(SectionPropDict) :
    PhiM = 0.9
    if SectionPropDict['Shape'] == 'H' :
        Mny = Minor_Axis_Flange_Local_Buckling_HSection(SectionPropDict)
    if SectionPropDict['Shape'] == 'L' :
        Mny = Minor_Axis_Leg_Local_Buckling_LSection(SectionPropDict)
    if SectionPropDict['Shape'] == 'T' :
        Mny = Minor_Axis_Flange_Local_Buckling_TSection(SectionPropDict)
    if SectionPropDict['Shape'] == '2L' :
        Mny = Minor_Axis_Flange_Local_Buckling_DoubleLSection(SectionPropDict)
    ΦMny = PhiM*Mny
    return ΦMny

def Minor_Axis_Flange_Local_Buckling_HSection(SectionPropDict) : 
    Syy = SectionPropDict['Syy']
    Zyy = SectionPropDict['Zyy']
    W = SectionPropDict['W']
    tf = SectionPropDict['tf']
    Mp = Fy*Zyy

    λf = W/(2*tf)
    λpf = 0.38*math.sqrt(E/Fy)
    λrf = 1.00*math.sqrt(E/Fy)

    if λf <= λpf :                                                   # Compact Flange
        Mny = min(Fy*Zyy, 1.6*Fy*Syy)
    elif λpf < λf <= λrf :                                           # Non-Compact Flange
        Mny = Mp-(Mp-0.7*Fy*Syy)*(λf-λpf)/(λrf-λpf)
    elif λrf < λf :                                                  # lender Flange
        Fcr = 0.7*E/λf**2
        Mny = Fcr*Syy
    return Mny

def Minor_Axis_Leg_Local_Buckling_LSection(SectionPropDict) : 
    W = SectionPropDict['W']
    tf = SectionPropDict['tf']
    Svmin = SectionPropDict['Svmin']

    λf = W/tf
    λp = 0.54*math.sqrt(E/Fy)
    λr = 0.91*math.sqrt(E/Fy)

    if λf <= λp : 
        Mnv = Svmin*Fy
        Mny = 1.5*Mnv
    if λp < λf <= λr : 
        Mny = Fy*Svmin*(2.43-1.72*(λf)*math.sqrt(Fy/E))
    if λr < λf : 
        Fcr = 0.71*E/λf**2
        Mny = Fcr*Svmin

    return Mny

def Minor_Axis_Flange_Local_Buckling_TSection(SectionPropDict) : 
    Syy = SectionPropDict['Syy']
    W = SectionPropDict['W']
    tf = SectionPropDict['tf']

    My = Fy*Syy
    Mp = My
    λf = W/(2*tf)
    λpf = 0.38*math.sqrt(E/Fy)
    λrf = 1.00*math.sqrt(E/Fy)

    if λf <= λpf :                                                  # Compression Flange Local Buckling
        Mny = My
    if λpf < λf <= λrf : 
        Syc = Syy
        Mny = min((Mp-(Mp-0.7*Fy*Syc)*(λf/λpf)/(λrf-λpf)),1.6*My)
    if λrf < λf : 
        Syc = Syy
        Mny = 0.7*E*Syc/(λf)**2

    return Mny

def Minor_Axis_Flange_Local_Buckling_DoubleLSection(SectionPropDict) : 
    Syy = SectionPropDict['Syy']
    W = SectionPropDict['W']
    tf = SectionPropDict['tf']

    My = Fy*Syy
    λf = W/tf
    λpf = 0.54*math.sqrt(E/Fy)
    λrf = 0.91*math.sqrt(E/Fy)

    if λf <= λpf :                                                  # Compression Flange Local Buckling
        Mny = My
    if λpf < λf <= λrf : 
        Syc = Syy
        Mny = Fy*Syc*(2.43-1.72*(λf)*math.sqrt(Fy/E))
    if λrf < λf : 
        Syc = Syy
        Fcr = 0.71*E/λf**2
        Mny = Fcr*Syc

    return Mny

In [15]:
def Calculate_Shear_Capacity(SectionPropDict) : 
    if SectionPropDict['Shape'] == 'H' :
        ΦVny, ΦVnx = Shear_Capacity_HSection(SectionPropDict)
    if SectionPropDict['Shape'] == 'L' :
        ΦVny, ΦVnx = Shear_Capacity_LSection(SectionPropDict)
    if SectionPropDict['Shape'] == 'T' :
        ΦVny, ΦVnx = Shear_Capacity_TSection(SectionPropDict)
    if SectionPropDict['Shape'] == '2L' :
        ΦVny, ΦVnx = Shear_Capacity_DoubleLSection(SectionPropDict)
    return ΦVny, ΦVnx
    
def Shear_Capacity_HSection(SectionPropDict) : 
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tw = SectionPropDict['tw']
    tf = SectionPropDict['tf']
    radi = SectionPropDict['radi']
    λw = (D-2*tf-2*radi)/tw
    λf = W/(2*tf)

    if λw <= 2.24*math.sqrt(E/Fy) : 
        Cv1 = 1.0
        PhiV = 1.0
    else : 
        if λw <= 1.1*math.sqrt(5.34*E/Fy) : 
            Cv1 = 1.0
            PhiV = 0.9
        else : 
            Cv1 = 1.1*math.sqrt(5.34*E/Fy)/λw
            PhiV = 0.9

    ΦVny = PhiV*0.6*Fy*tw*D*Cv1      # Shear Strength of Webs

    if λf <= 1.1*math.sqrt(1.2*E/Fy) : 
        Cv2 = 1.0
        PhiV = 0.9
    elif 1.1*math.sqrt(1.2*E/Fy) < λf <= 1.37*math.sqrt(1.2*E/Fy) :
        Cv2 = 1.1*math.sqrt(1.2*E/Fy)/λf
        PhiV = 0.9
    else : 
        Cv2 = 1.51*1.2*E/(λf**2*Fy)
        PhiV = 0.9

    ΦVnx = PhiV*0.6*Fy*W*tf*Cv2*2    # Shear Strength of Flanges

    return ΦVny, ΦVnx

def Shear_Capacity_LSection(SectionPropDict) : 
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tw = SectionPropDict['tw']
    tf = SectionPropDict['tf']
    radi = SectionPropDict['radi']
    λw = D/tw
    λf = W/tf

    if λw <= 1.1*math.sqrt(1.2*E/Fy) : 
        Cv2 = 1.0
        PhiV = 0.9
    elif 1.1*math.sqrt(1.2*E/Fy) < λw <= 1.37*math.sqrt(1.2*E/Fy) :
        Cv2 = 1.1*math.sqrt(1.2*E/Fy)/λw
        PhiV = 0.9
    else : 
        Cv2 = 1.51*1.2*E/(λw**2*Fy)
        PhiV = 0.9

    ΦVny = PhiV*0.6*Fy*tw*D*Cv2      # Shear Strength of Webs

    if λf <= 1.1*math.sqrt(1.2*E/Fy) : 
        Cv2 = 1.0
        PhiV = 0.9
    elif 1.1*math.sqrt(1.2*E/Fy) < λf <= 1.37*math.sqrt(1.2*E/Fy) :
        Cv2 = 1.1*math.sqrt(1.2*E/Fy)/λf
        PhiV = 0.9
    else : 
        Cv2 = 1.51*1.2*E/(λf**2*Fy)
        PhiV = 0.9

    ΦVnx = PhiV*0.6*Fy*W*tf*Cv2       # Shear Strength of Flanges

    return ΦVny, ΦVnx

def Shear_Capacity_TSection(SectionPropDict) : 
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tw = SectionPropDict['tw']
    tf = SectionPropDict['tf']
    radi = SectionPropDict['radi']
    λw = D/tw
    λf = W/(2*tf)

    if λw <= 1.1*math.sqrt(5*E/Fy) : 
        Cv2 = 1.0
        PhiV = 0.9
    elif 1.1*math.sqrt(5*E/Fy) < λw <= 1.37*math.sqrt(5*E/Fy) :
        Cv2 = 1.1*math.sqrt(5*E/Fy)/λw
        PhiV = 0.9
    else : 
        Cv2 = 1.51*5*E/(λw**2*Fy)
        PhiV = 0.9

    ΦVny = PhiV*0.6*Fy*tw*D*Cv2      # Shear Strength of Webs

    if λf <= 1.1*math.sqrt(1.2*E/Fy) : 
        Cv2 = 1.0
        PhiV = 0.9
    elif 1.1*math.sqrt(1.2*E/Fy) < λf <= 1.37*math.sqrt(1.2*E/Fy) :
        Cv2 = 1.1*math.sqrt(1.2*E/Fy)/λf
        PhiV = 0.9
    else : 
        Cv2 = 1.51*1.2*E/(λf**2*Fy)
        PhiV = 0.9

    ΦVnx = PhiV*0.6*Fy*W*tf*Cv2       # Shear Strength of Flanges

    return ΦVny, ΦVnx

def Shear_Capacity_DoubleLSection(SectionPropDict) : 
    D = SectionPropDict['D']
    W = SectionPropDict['W']
    tw = SectionPropDict['tw']
    tf = SectionPropDict['tf']
    radi = SectionPropDict['radi']
    λw = D/tw
    λf = W/tf

    if λw <= 1.1*math.sqrt(1.2*E/Fy) : 
        Cv2 = 1.0
        PhiV = 0.9
    elif 1.1*math.sqrt(1.2*E/Fy) < λw <= 1.37*math.sqrt(1.2*E/Fy) :
        Cv2 = 1.1*math.sqrt(1.2*E/Fy)/λw
        PhiV = 0.9
    else : 
        Cv2 = 1.51*1.2*E/(λw**2*Fy)
        PhiV = 0.9

    ΦVny = PhiV*0.6*Fy*tw*D*Cv2*2      # Shear Strength of Webs

    if λf <= 1.1*math.sqrt(1.2*E/Fy) : 
        Cv2 = 1.0
        PhiV = 0.9
    elif 1.1*math.sqrt(1.2*E/Fy) < λf <= 1.37*math.sqrt(1.2*E/Fy) :
        Cv2 = 1.1*math.sqrt(1.2*E/Fy)/λf
        PhiV = 0.9
    else : 
        Cv2 = 1.51*1.2*E/(λf**2*Fy)
        PhiV = 0.9

    ΦVnx = PhiV*0.6*Fy*W*tf*Cv2*2       # Shear Strength of Flanges

    return ΦVny, ΦVnx

In [16]:
def Calculate_Combined_Ratio(ForceDict, Pc, Mcx, Mcy) : 
    if ForceDict['Pu']/Pc >= 0.2 : 
        CR = abs(ForceDict['Pu'])/Pc+(8/9)*(abs(ForceDict['Mux'])/Mcx+abs(ForceDict['Muy'])/Mcy)
    else : 
        CR = abs(ForceDict['Pu'])/(2*Pc)+(abs(ForceDict['Mux'])/Mcx+abs(ForceDict['Muy'])/Mcy)
    return CR

In [17]:
ColumnApplicationListDict = copy.deepcopy(ApplicationListDict)

def Calculate_Column_Capacity(SectionPropDict, ColumnUnbracedSegment, ColumnForceDict, ColumnLength) : 
    elemDict= {}
    SR = Calculate_Slenderness_Ratio(SectionPropDict,ColumnUnbracedSegment,ColumnLength)
    ΦPn1 = Calculate_Tensile_Strength(SectionPropDict)
    ΦPn2 = Calculate_Flexural_Buckling(SectionPropDict,ColumnUnbracedSegment,ColumnLength)
    ΦPn3 = Calculate_Flexural_Torsional_Buckling(SectionPropDict,ColumnUnbracedSegment,ColumnLength)
    if ColumnForceDict['Pu'] < 0 : 
        ΦPn = ΦPn1
    else : 
        ΦPn = min(ΦPn2,ΦPn3)
    ΦMnx1 = Calculate_Lateral_Torsional_Buckling(SectionPropDict,ColumnUnbracedSegment,ColumnLength)
    ΦMnx2 = Calculate_Local_Buckling(SectionPropDict)
    ΦMnx3 = Calculate_Tension_Element_Yielding(SectionPropDict)
    ΦMnx = min(ΦMnx1, ΦMnx2, ΦMnx3)
    ΦMny = Calculate_Minor_Axis_Flange_Local_Buckling(SectionPropDict)
    ΦVny, ΦVnx = Calculate_Shear_Capacity(SectionPropDict)
    CR = Calculate_Combined_Ratio(ColumnForceDict, ΦPn, ΦMnx, ΦMny)

    ratioDict = {
        "SectionName": SectionPropDict['SectionName'],
        "SR": SR,
        "ΦPn": ΦPn, 
        "ΦMnx": ΦMnx, 
        "ΦMny": ΦMny, 
        "ΦVny": ΦVny, 
        "ΦVnx": ΦVnx,
        "pRatio": abs(ColumnForceDict["Pu"])/ΦPn,
        "mxRatio": abs(ColumnForceDict["Mux"])/ΦMnx,
        "myRatio": abs(ColumnForceDict["Muy"])/ΦMny,
        "vyRatio": abs(ColumnForceDict["Vuy"])/ΦVny,
        "vxRatio": abs(ColumnForceDict["Vux"])/ΦVnx,
        "CR": CR,
    }
    elemDict["ratioDict"] = ratioDict
    
    if ratioDict["pRatio"]<1 and ratioDict["mxRatio"]<1 and ratioDict["myRatio"]<1 and ratioDict["vxRatio"]<1 and ratioDict["vyRatio"]<1 and ratioDict["CR"]<1 :
        elemDict["chk_result"] = 'OK'
    else : 
        elemDict["chk_result"] = 'NG'

    if ratioDict["SR"]>200 :
        elemDict["Remark"] = "effective slenderness ratio, Lc/r , preferably should not exceed 200"

    return elemDict

ColumnCalculatedResult = Calculate_Column_Capacity(ColumnSection, ColumnUnbracedSegment, ColumnForceDict, ColumnLength)

def Select_Applicable_Column(ApplicationListDict) :
    ResultList = []
    subdicts = [value for value in ApplicationListDict.values() if isinstance(value, dict)]
    for subdict in subdicts : 
        ResultList.append(Calculate_Column_Capacity(subdict, ColumnUnbracedSegment, ColumnForceDict, ColumnLength))
    return ResultList

CalculatedApplicableColumn = Select_Applicable_Column(ColumnApplicationListDict)

def Delete_NG_Element(CalculatedApplicableElement) : 
    ResultList = []
    for subdict in CalculatedApplicableElement :
        if subdict["chk_result"] == 'OK' : 
            ResultList.append(subdict)
    return ResultList

Selected_Column_List = Delete_NG_Element(CalculatedApplicableColumn)

print(ColumnCalculatedResult)       # 적용 부재 Ratio S검토
print(CalculatedApplicableColumn)
print(Selected_Column_List)          # OK 나오는 Column List

{'ratioDict': {'SectionName': '2L-90x90x8', 'SR': 251.43425613769944, 'ΦPn': 71.21322905419662, 'ΦMnx': 10959.81056856955, 'ΦMny': 12206.35957894737, 'ΦVny': 213.84000000000003, 'ΦVnx': 213.84000000000003, 'pRatio': 0.2921507741794213, 'mxRatio': 0.7579510565467932, 'myRatio': 0.8696286498316804, 'vyRatio': 0.03146745230078563, 'vxRatio': 0.01421343060231949, 'CR': 1.7388882909602867}, 'chk_result': 'NG', 'Remark': 'effective slenderness ratio, Lc/r , preferably should not exceed 200'}
[{'ratioDict': {'SectionName': 'L-90x90x8', 'SR': 296.43425613769944, 'ΦPn': 56.27552891730549, 'ΦMnx': 6919.444841527854, 'ΦMny': 4467.8370700192345, 'ΦVny': 106.92000000000002, 'ΦVnx': 106.92000000000002, 'pRatio': 0.36969887978435645, 'mxRatio': 1.2005298387732743, 'myRatio': 2.375869986672164, 'vyRatio': 0.06293490460157126, 'vxRatio': 0.02842686120463898, 'CR': 3.5487209468469683}, 'chk_result': 'NG', 'Remark': 'effective slenderness ratio, Lc/r , preferably should not exceed 200'}, {'ratioDict': {'

In [18]:
GirderApplicationListDict = copy.deepcopy(ColumnApplicationListDict)
BeamApplicationListDict = copy.deepcopy(ColumnApplicationListDict)

def Calculate_Girder_Beam_Capacity(SectionPropDict, GirderUnbracedSegment, GirderForceDict, GirderLength, BeamForceDict, BeamLength, BeamAttachedGirderLength, Element) : 
    elemDict = {}
    if Element == 'Girder' :
        SR = Calculate_Slenderness_Ratio(SectionPropDict,GirderUnbracedSegment,GirderLength)
        ΦPn1 = Calculate_Tensile_Strength(SectionPropDict)
        ΦPn2 = Calculate_Flexural_Buckling(SectionPropDict,GirderUnbracedSegment,GirderLength)
        ΦPn3 = Calculate_Flexural_Torsional_Buckling(SectionPropDict,GirderUnbracedSegment,GirderLength)
        if GirderForceDict['Pu'] < 0 : 
            ΦPn = ΦPn1
        else : 
            ΦPn = min(ΦPn2, ΦPn3)
        ΦMnx1 = Calculate_Lateral_Torsional_Buckling(SectionPropDict,GirderUnbracedSegment,GirderLength)
        ΦMnx2 = Calculate_Local_Buckling(SectionPropDict)
        ΦMnx3 = Calculate_Tension_Element_Yielding(SectionPropDict)
        ΦMnx = min(ΦMnx1, ΦMnx2, ΦMnx3)
        ΦMny = Calculate_Minor_Axis_Flange_Local_Buckling(SectionPropDict)
        ΦVny, ΦVnx = Calculate_Shear_Capacity(SectionPropDict)
        CR = Calculate_Combined_Ratio(GirderForceDict, ΦPn, ΦMnx, ΦMny)

        ratioDict = {
            "SectionName": SectionPropDict['SectionName'],
            "SR": SR,
            "ΦPn": ΦPn, 
            "ΦMnx": ΦMnx, 
            "ΦMny": ΦMny, 
            "ΦVny": ΦVny, 
            "ΦVnx": ΦVnx,
            "pRatio": abs(GirderForceDict["Pu"])/ΦPn,
            "mxRatio": abs(GirderForceDict["Mux"])/ΦMnx,
            "myRatio": abs(GirderForceDict["Muy"])/ΦMny,
            "vyRatio": abs(GirderForceDict["Vuy"])/ΦVny,
            "vxRatio": abs(GirderForceDict["Vux"])/ΦVnx,
            "CR": CR,
        }
        elemDict["ratioDict"] = ratioDict
        
        if ratioDict["pRatio"]<1 and ratioDict["mxRatio"]<1 and ratioDict["myRatio"]<1 and ratioDict["vyRatio"]<1 and ratioDict["vxRatio"]<1 and ratioDict["CR"]<1 :
            elemDict["chk_result"] = 'OK'
        else : 
            elemDict["chk_result"] = 'NG'

        if ratioDict["SR"]>200 :
            elemDict["Remark"] = "effective slenderness ratio, Lc/r , preferably should not exceed 200"

    if Element == 'Beam' : 
        BeamUnbracedSegment = 1               # Sub-Beam은 횡좌굴 잡지 않는다고 가정한 부분
        SR = Calculate_Slenderness_Ratio(SectionPropDict,BeamUnbracedSegment,BeamLength)
        ## Brace 용도 / Start
        Lbr = BeamAttachedGirderLength/GirderUnbracedSegment
        Pr = GirderForceDict['Pu']
        Mr = GirderForceDict['Muy']
        Cd = 2
        ho = GirderSection['D']
        Pbr = 0.01*Pr+0.02*(Mr*Cd/ho)
        βbr = 1/0.75*(8*Pr/Lbr)+1/0.75*(10*Mr*Cd/(Lbr*ho))
        Stiffness = E*(SectionPropDict['Area'])/BeamLength
        Stiffness_Ratio = βbr/Stiffness
        ## Brace 용도 / End
        ΦPn1 = Calculate_Tensile_Strength(SectionPropDict)
        ΦPn2 = Calculate_Flexural_Buckling(SectionPropDict,BeamUnbracedSegment,BeamLength)
        ΦPn3 = Calculate_Flexural_Torsional_Buckling(SectionPropDict,BeamUnbracedSegment,BeamLength)
        if BeamForceDict['Pu'] < 0 : 
            ΦPn = ΦPn1
        else : 
            ΦPn = min(ΦPn2, ΦPn3)
        ΦMnx1 = Calculate_Lateral_Torsional_Buckling(SectionPropDict,BeamUnbracedSegment,BeamLength)
        ΦMnx2 = Calculate_Local_Buckling(SectionPropDict)
        ΦMnx3 = Calculate_Tension_Element_Yielding(SectionPropDict)
        ΦMnx = min(ΦMnx1, ΦMnx2, ΦMnx3)
        ΦMny = Calculate_Minor_Axis_Flange_Local_Buckling(SectionPropDict)
        ΦVny, ΦVnx = Calculate_Shear_Capacity(SectionPropDict)
        CR = Calculate_Combined_Ratio(BeamForceDict, ΦPn, ΦMnx, ΦMny)

        ratioDict = {
            "SectionName": SectionPropDict['SectionName'],
            "SR": SR,
            "ΦPn": ΦPn, 
            "ΦMnx": ΦMnx, 
            "ΦMny": ΦMny, 
            "ΦVny": ΦVny, 
            "ΦVnx": ΦVnx,
            "pRatio": abs(max(BeamForceDict["Pu"],Pbr))/ΦPn,
            "mxRatio": abs(BeamForceDict["Mux"])/ΦMnx,
            "myRatio": abs(BeamForceDict["Muy"])/ΦMny,
            "vyRatio": abs(BeamForceDict["Vuy"])/ΦVny,
            "vxRatio": abs(BeamForceDict["Vux"])/ΦVnx,
            "CR": CR,
        }
        elemDict["ratioDict"] = ratioDict
        
        if ratioDict["pRatio"]<1 and ratioDict["mxRatio"]<1 and ratioDict["myRatio"]<1 and ratioDict["vyRatio"]<1 and ratioDict["vxRatio"]<1 and ratioDict["CR"]<1 and Stiffness_Ratio<1:
            elemDict["chk_result"] = 'OK'
        else : 
            elemDict["chk_result"] = 'NG'

        if ratioDict["SR"]>200 :
            elemDict["Remark"] = "effective slenderness ratio, Lc/r , preferably should not exceed 200"

    return elemDict
    
GirderCalculatedResult = Calculate_Girder_Beam_Capacity(GirderSection, GirderUnbracedSegment, GirderForceDict, GirderLength, BeamForceDict, BeamLength, BeamAttachedGirderLength, 'Girder')
BeamCalculatedResult = Calculate_Girder_Beam_Capacity(BeamSection, GirderUnbracedSegment, GirderForceDict, GirderLength, BeamForceDict, BeamLength, BeamAttachedGirderLength, 'Beam')

def Select_Applicable_Girder_Beam(ApplicationListDict, GirderUnbracedSegment, GirderForceDict, GirderLength, BeamForceDict, BeamLength, BeamAttachedGirderLength, Element) :
    ResultList = []
    subdicts = [value for value in ApplicationListDict.values() if isinstance(value, dict)]
    for subdict in subdicts : 
        ResultList.append(Calculate_Girder_Beam_Capacity(subdict, GirderUnbracedSegment, GirderForceDict, GirderLength, BeamForceDict, BeamLength, BeamAttachedGirderLength, Element))
    return ResultList

CalculatedApplicableGirder = Select_Applicable_Girder_Beam(GirderApplicationListDict, GirderUnbracedSegment, GirderForceDict, GirderLength, BeamForceDict, BeamLength, BeamAttachedGirderLength, 'Girder')
CalculatedApplicableBeam = Select_Applicable_Girder_Beam(BeamApplicationListDict, GirderUnbracedSegment, GirderForceDict, GirderLength, BeamForceDict, BeamLength, BeamAttachedGirderLength, 'Beam')
SelectedApplicableGirder = Delete_NG_Element(CalculatedApplicableGirder)
SelectedApplicableBeam = Delete_NG_Element(CalculatedApplicableBeam)

print(GirderCalculatedResult)
print(BeamCalculatedResult)
print(SelectedApplicableGirder)
print(SelectedApplicableBeam)

{'ratioDict': {'SectionName': 'L-90x90x8', 'SR': 296.43425613769944, 'ΦPn': 56.27552891730549, 'ΦMnx': 6919.444841527854, 'ΦMny': 4467.8370700192345, 'ΦVny': 106.92000000000002, 'ΦVnx': 106.92000000000002, 'pRatio': 0.8893741376211026, 'mxRatio': 1.086503349933487, 'myRatio': 1.6356908019406666, 'vyRatio': 0.9352787130564907, 'vxRatio': 0.46763935652824534, 'CR': 3.30910227262035}, 'chk_result': 'NG', 'Remark': 'effective slenderness ratio, Lc/r , preferably should not exceed 200'}
{'ratioDict': {'SectionName': 'UB-203x102x23', 'SR': 224.56434784166277, 'ΦPn': 106.81551239483564, 'ΦMnx': 29556.176056278044, 'ΦMny': 12539.71818, 'ΦVny': 254.81279999999998, 'ΦVnx': 280.6293600000001, 'pRatio': 0.4685649010884662, 'mxRatio': 0.25436308085609394, 'myRatio': 0.5827882170155757, 'vyRatio': 0.3924449635183162, 'vxRatio': 0.17817095117916382, 'CR': 1.212699388085506}, 'chk_result': 'NG', 'Remark': 'effective slenderness ratio, Lc/r , preferably should not exceed 200'}
[{'ratioDict': {'SectionN